# C-SVM

C-SVM is a binary classifier that use the parameter $C$ to set the number of support vectors by setting the margin width. Before we go into the C-SVM implementation, we need to restore the dataset.

In [23]:
from sklearn.datasets import load_iris
import numpy as np
from sklearn.model_selection import train_test_split
import cvxopt

iris = load_iris()
data_set = iris.data
labels = iris.target

data_set = data_set[labels!=2]
labels = labels[labels!=2]

train_data_set, test_data_set, train_labels, test_labels = train_test_split(
    data_set, labels, test_size=0.2, random_state=15)

train_labels[train_labels<1] = -1
test_labels[test_labels<1] = -1

objects_count = len(train_labels)

In this example we use the linear kernel $K=XX^{T}$:

In [24]:
def build_kernel(data_set, kernel_type='linear'):
    kernel = np.dot(data_set, data_set.T)
    if kernel_type == 'rbf':
        sigma = 1.0
        objects_count = len(data_set)
        b = np.ones((len(data_set), 1))
        kernel -= 0.5 * (np.dot((np.diag(kernel)*np.ones((1, objects_count))).T, b.T)
                         + np.dot(b, (np.diag(kernel) * np.ones((1, objects_count))).T.T))
        kernel = np.exp(kernel / (2. * sigma ** 2))
    return kernel

The training phase is similar to what we had in the introdction, but we added the $C$ parameter.

In [25]:
def train(train_data_set, train_labels, kernel_type='linear', C=10, threshold=1e-5):
    kernel = build_kernel(train_data_set, kernel_type=kernel_type)

    P = train_labels * train_labels.transpose() * kernel
    q = -np.ones((objects_count, 1))
    G = np.concatenate((np.eye(objects_count), -np.eye(objects_count)))
    h = np.concatenate((C * np.ones((objects_count, 1)), np.zeros((objects_count, 1))))

    A = train_labels.reshape(1, objects_count)
    A = A.astype(float)
    b = 0.0

    sol = cvxopt.solvers.qp(cvxopt.matrix(P), cvxopt.matrix(q), cvxopt.matrix(G), cvxopt.matrix(h), cvxopt.matrix(A), cvxopt.matrix(b))

    lambdas = np.array(sol['x'])

    support_vectors_id = np.where(lambdas > threshold)[0]
    vector_number = len(support_vectors_id)
    support_vectors = train_data_set[support_vectors_id, :]

    lambdas = lambdas[support_vectors_id]
    targets = train_labels[support_vectors_id]

    b = np.sum(targets)
    for n in range(vector_number):
        b -= np.sum(lambdas * targets * np.reshape(kernel[support_vectors_id[n], support_vectors_id], (vector_number, 1)))
    b /= len(lambdas)

    return lambdas, support_vectors, support_vectors_id, b, targets, vector_number

Finally, we get the prediction of the C-SVM classifier. Please keep in mind that we can get the soft margin values by removing ``np.sign``:

In [26]:
def classify_linear(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id):
    kernel = build_kernel(train_data_set)
    y = np.zeros((np.shape(test_data_set)[0], 1))
    for j in range(np.shape(test_data_set)[0]):
        for i in range(vector_number):
            y[j] += lambdas[i] * targets[i] * kernel[j, i]
        y[j] += b
    return np.sign(y)

The predictions are kept in ``prediction``:

In [27]:
lambdas, support_vectors, support_vectors_id, b, targets, vector_number = train(train_data_set, train_labels, kernel_type='linear')
predicted = classify_linear(test_data_set, train_data_set, lambdas, targets, b, vector_number, support_vectors, support_vectors_id)
predicted = list(predicted.astype(int))

     pcost       dcost       gap    pres   dres
 0: -1.1546e+00 -1.5602e+03  4e+03  3e-01  7e-14
 1:  1.3116e+00 -8.6134e+01  1e+02  8e-03  5e-14
 2:  1.1791e+00 -2.2936e+00  4e+00  8e-05  4e-15
 3:  1.6689e-01 -2.9221e-01  5e-01  2e-16  3e-15
 4:  6.1566e-03 -5.8993e-02  7e-02  2e-16  2e-15
 5: -1.0543e-02 -1.8182e-02  8e-03  2e-16  5e-16
 6: -1.2195e-02 -1.5679e-02  3e-03  2e-16  3e-16
 7: -1.3639e-02 -1.6237e-02  3e-03  2e-16  2e-16
 8: -1.4910e-02 -1.5084e-02  2e-04  2e-16  3e-16
 9: -1.4930e-02 -1.5062e-02  1e-04  1e-16  3e-16
10: -1.4985e-02 -1.4990e-02  6e-06  2e-16  2e-16
11: -1.4987e-02 -1.4987e-02  1e-07  2e-16  3e-16
12: -1.4987e-02 -1.4987e-02  2e-09  2e-16  2e-16
Optimal solution found.


The accuracy is:

In [28]:
from sklearn.metrics import accuracy_score

print(accuracy_score(predicted, test_labels))

0.55
